In [20]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/GuoRubing/Google Drive/Python/uams_reports_test/sap2.csv")


In [21]:
df.head()

,PO Number,PO Line,Cont Ind,Contract,Cont Type,Invoice Date,Invoice No,Vendor Reference,Date PO Created,Total Cost per Line,...,Manufacturer Name,UNSPSC code,UNSPSC Desc,Purch Gp,Buyer Name,Case No,SQ Cart No,Req No,Req Line,PO Unit Price
0,4.500134e+09,300.0,NaN,NONC1500-M,NaN,1/22/2016,5.109316e+09,1.093508e+09,1/19/2016,260.81,...,MEDLINE INDUSTRIES INC,42182201.0,ELECTRONIC MEDICAL THERMOMETERS,WB,Janice Carter,NaN,NaN,12101312.0,10.0,260.81
1,4.500134e+09,3.0,NaN,NaN,NaN,1/21/2016,5.109316e+09,5.109316e+09,1/19/2016,34.2,...,FISHER HEALTHCARE,41112220.0,LABORATORY FREEZER OR REFRIGERATOR THERMOMETERS,WB,Janice Carter,NaN,NaN,12101192.0,10.0,5.70
2,4.500134e+09,1.0,NaN,NaN,NaN,2/2/2016,5.109324e+09,5.109324e+09,1/27/2016,5.7,...,FISHER HEALTHCARE,41112220.0,LABORATORY FREEZER OR REFRIGERATOR THERMOMETERS,WB,Janice Carter,NaN,NaN,12105105.0,10.0,5.70
3,4.500135e+09,285.0,NaN,NONC1500-M,NaN,2/23/2016,5.109339e+09,1.094749e+09,2/16/2016,260.81,...,MEDLINE INDUSTRIES INC,42182201.0,ELECTRONIC MEDICAL THERMOMETERS,WB,Janice Carter,NaN,NaN,12114089.0,10.0,260.81
4,4.500135e+09,2.0,NaN,NONC1500-M,NaN,2/29/2016,5.109344e+09,1.094976e+09,2/26/2016,22.32,...,MEDLINE INDUSTRIES INC,42182201.0,ELECTRONIC MEDICAL THERMOMETERS,LB,LUM Buyer,NaN,NaN,12119375.0,26.0,1.86


In [22]:
df_sap = df[df['Qty Paid'] != 0]

# Last Tab 
s1 = df_sap['Manufacturer Part No']
s2 = df_sap['U O M']
df_sap['MPN-UOM'] = s1.str.cat(s2, sep = '-')

s3 = df_sap['MPN-UOM']
s4 = df_sap['Date PO Created']
s5 = df_sap['Description']
s6 = df_sap['Unit Price less Dist Fee']
pd.concat([s3, s4, s5, s6], axis=1).head()



,MPN-UOM,Date PO Created,Description,Unit Price less Dist Fee
0,MDS9950-CS,1/19/2016,THERMOMETER ORAL ELEC DGTL SCALE 144/CS,256.32
1,13-201-966-EA,1/19/2016,THERMOMETER 15105S FRZ-GUIDE -40 TO +80,5.70
2,13-201-966-EA,1/27/2016,THERMOMETER 15105S FRZ-GUIDE -40 TO +80,5.70
3,MDS9950-CS,2/16/2016,THERMOMETER ORAL ELEC DGTL SCALE 144/CS,256.32
4,MDS9950-EA,2/26/2016,THERMOMETER ORAL ELEC DGTL SCALE,1.83


In [23]:
df_last = pd.concat([s3, s4, s5, s6], axis=1).head()
df_last = df_last.sort_values(['Date PO Created', 'MPN-UOM'],ascending = [0,1])

df_last = df_last.drop_duplicates(subset = ['MPN-UOM'], keep = 'first')
#df_last.loc['Last Price'] = df_last['Unit Price less Dist Fee']
df_last = df_last.rename(columns = {'Unit Price less Dist Fee':'Last Price'})
df_last = df_last.rename(columns = {'Description':'Last Description'})
df_last

,MPN-UOM,Date PO Created,Last Description,Last Price
4,MDS9950-EA,2/26/2016,THERMOMETER ORAL ELEC DGTL SCALE,1.83
3,MDS9950-CS,2/16/2016,THERMOMETER ORAL ELEC DGTL SCALE 144/CS,256.32
2,13-201-966-EA,1/27/2016,THERMOMETER 15105S FRZ-GUIDE -40 TO +80,5.70


In [24]:

df_sum = pd.merge(df_sap, df_last, left_on = 'MPN-UOM', right_on = 'MPN-UOM', how='left')
df_sum.to_csv('file_name.csv')
df_sum['Total'] = df_sum['Last Price'] * df_sum['Qty Paid']
table = pd.pivot_table(df_sum, values=['Qty Paid', 'Total'], index=['Manufacturer Name', 'MPN-UOM', 'Vendor Mat Number', 'Last Description', 'Last Price', 'U O M'], columns = [], aggfunc=np.sum)
table

Qty Paid  \
Manufacturer Name      MPN-UOM       Vendor Mat Number Last Description                        Last Price U O M             
FISHER HEALTHCARE      13-201-966-EA 13-201-966        THERMOMETER 15105S FRZ-GUIDE -40 TO +80 5.70       EA          9.0   
MEDLINE INDUSTRIES INC MDS9950-CS    MDS9950           THERMOMETER ORAL ELEC DGTL SCALE 144/CS 256.32     CS          2.0   
                       MDS9950-EA    MDS9950H          THERMOMETER ORAL ELEC DGTL SCALE        1.83       EA        250.0   

                                                                                                                  Total  
Manufacturer Name      MPN-UOM       Vendor Mat Number Last Description                        Last Price U O M          
FISHER HEALTHCARE      13-201-966-EA 13-201-966        THERMOMETER 15105S FRZ-GUIDE -40 TO +80 5.70       EA      51.30  
MEDLINE INDUSTRIES INC MDS9950-CS    MDS9950           THERMOMETER ORAL ELEC DGTL SCALE 144/CS 256.32     CS     512.64  
                       MDS9950-EA    MDS9950H          THERMOMETER ORAL ELEC DGTL SCALE        1.83       EA     457.50

In [25]:
table['Total'].sum()

1021.4400000000001